In [ ]:
import glob
import japanize_matplotlib
import os
import pandas as pd
import random
import sys
import matplotlib as mpl
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [ ]:
def return_fixed_class(BenchMark="bt", Processes=[1, 2, 4, 8, 16, 32, 64, 128, 256], FixedBenchMarkClass="C"):
        path = './'
        
        # fixed_Class にはベンチマーククラスFixedBenchMarkClassで実行プロセス数がProcessesに該当するものの結果が入る
        fixed_Class = list()

        for process in Processes:
            file_name = ("pprof_"+BenchMark+FixedBenchMarkClass+str(process)+".csv")
            file_path = path+file_name
            if (os.path.exists(file_path) and os.stat(file_path).st_size != 0):
                data_frame = pd.read_csv(path+file_name)
                data_frame = data_frame.set_index(['Name'])
                fixed_Class.append(data_frame.rename(columns = {'#Call': process}).sort_index())
        return(fixed_Class)
    

In [ ]:
BenchMark_name = "cg"
cg_fixed_class = return_fixed_class(BenchMark=BenchMark_name)
cg_fixed_process = return_fixed_process(BenchMark=BenchMark_name)
summary_fixed_class = pd.concat(cg_fixed_class, axis=1)

columns = summary_fixed_class.columns.to_numpy()

index = summary_fixed_class.index.to_numpy()

dict_summary_fixed_class = {"benchmarks": columns}
for index_name in index:
    dict_summary_fixed_class[index_name] = summary_fixed_class.T[index_name].to_numpy()

In [ ]:
def return_fixed_process(BenchMark="bt", BenchMarkClasses=["S", "W", "A", "B", "C", "D"], FixedProcess=32):
    path = './'

    # fixed_process には実行プロセス数が64でベンチマーククラスがA ~ Dまでの結果が入る
    fixed_process = list()

    for bench_mark_class in BenchMarkClasses:
        file_name = ("pprof_"+BenchMark+bench_mark_class+str(FixedProcess)+".csv")
        file_path = path+file_name
        if (os.path.exists(file_path) and os.stat(file_path).st_size != 0):
            data_frame = pd.read_csv(path+file_name)
            data_frame = data_frame.set_index(['Name'])
            fixed_process.append(data_frame.rename(columns = {'#Call': bench_mark_class}).sort_index())
    return(fixed_process)
       

In [ ]:
# x軸, y軸ともに対数にしようとしてみる

# 対数化に必要なので導入
import sklearn.preprocessing as sp
from sklearn.metrics import r2_score

# 常用対数の逆関数
def inverter_log10_func(x):
    return 10**x

# transformer_log10 : 渡された値に対して一括して常用対数をとるメソッド?
transformer_log10 = sp.FunctionTransformer(func=np.log10, inverse_func=inverter_log10_func)

# 引数として学習用のx, yがあり、線形関数として学習したモデルを返す
def return_lr_lin(input_x, input_y):
    lr = LinearRegression()
    lr.fit(input_x, input_y)
    return lr

# 引数として学習用のx, yをとり、非線形関数として学習したモデルを返す
def return_lr_log10(input_x, input_y):
    # N_train_log10 : 学習用のデータに常用対数をとったもの
    x_train_log10 = transformer_log10.transform(input_x)
    y_train_log10 = transformer_log10.transform(input_y)
    # 線形回帰モデルの生成
    lr = LinearRegression()
    # 学習
    lr.fit(x_train_log10, y_train_log10)
    return lr

# 引数としてテスト用のx, y、線形関数として学習したモデルlrをとり、スコアを返す
def return_lr_lin_score(input_x, input_y, lr):
    y_predict = lr.predict(input_x)
    _r2_score = r2_score(input_y, y_predict)
    return _r2_score

# 引数としてテスト用のx, y、非線形関数として学習したモデルlrをとり、スコアを返す
def return_lr_log10_score(input_x, input_y, lr):
    # N_predict : モデル式を引くためのデータ, N_predict_log10 : モデル式を引くためのデータに常用対数をとったもの
    input_x_log10 = transformer_log10.transform(input_x)
    y_predict_log10 = lr.predict(input_x_log10)
    y_predict = transformer_log10.inverse_transform(y_predict_log10)
    _r2_score = r2_score(input_y, y_predict)
    return _r2_score